In [5]:
from fastai.text.all import *
import gc

In [2]:
torch.cuda.is_available()

True

#### Fine-tuning a language model on Rotten Tomatoes Movie Reviews
Data source: https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/overview

In [3]:
from IPython.display import Image
Image(url='https://docs.fast.ai/images/ulmfit.png')

In [3]:
path = Path("..") / "data/sample/"

In [5]:
df = pd.read_csv(path/"data.csv", nrows=10000)

In [4]:
df = pd.read_csv(path/"data.csv")
sample_df = df.sample(frac=0.02)

In [6]:
del df
gc.collect()

1406

In [6]:
BATCH_SIZE = 16
dls_lm = TextDataLoaders.from_df(df, path=path, text_col=0, label_col=1, is_lm=True, bs=BATCH_SIZE, valid_pct=0.1)

/home/lazylearner/anaconda3/envs/amazonreviews/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [7]:
BATCH_SIZE = 16
dls_lm = TextDataLoaders.from_df(sample_df, path=path, text_col=0, label_col=1, is_lm=True, bs=BATCH_SIZE, valid_pct=0.1)

/home/lazylearner/anaconda3/envs/amazonreviews/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [8]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj my wife dreams out loud of one republic , but not of me . xxmaj thanks man . xxmaj you stole my girl . \n\n xxmaj good xxmaj album . xxbos xxmaj an unabashed xxmaj kinks fan for a long time now , i wondered when an enterprising producer would mine the xxmaj kinks xxunk for one of these tribute compilations . xxmaj jim xxmaj pitt 's xxmaj this xxmaj","xxmaj my wife dreams out loud of one republic , but not of me . xxmaj thanks man . xxmaj you stole my girl . \n\n xxmaj good xxmaj album . xxbos xxmaj an unabashed xxmaj kinks fan for a long time now , i wondered when an enterprising producer would mine the xxmaj kinks xxunk for one of these tribute compilations . xxmaj jim xxmaj pitt 's xxmaj this xxmaj is"
1,"and beauty . xxmaj it is a pleasure to write with them and they are suitable for all fountain pens . i bought a dozen of different colors . xxmaj this is a wonderful green - blue color , very subtle and elegant . xxmaj the bottle is superb . xxmaj using such an ink , everyone could personalize his / her writing . xxbos xxmaj absolutely great cable charger . i","beauty . xxmaj it is a pleasure to write with them and they are suitable for all fountain pens . i bought a dozen of different colors . xxmaj this is a wonderful green - blue color , very subtle and elegant . xxmaj the bottle is superb . xxmaj using such an ink , everyone could personalize his / her writing . xxbos xxmaj absolutely great cable charger . i love"
2,"your head . "" xxunk y xxmaj sigue "" is a lovely salsa influenced track which again is so catchy will drive you nuts . xxmaj but my favourite track is the perfectly executed "" xxunk a xxmaj poco "" , xxmaj estopa 's view on smoking . xxmaj taking from the point of a smoker it describes his feelings and why he does it and what it is doing it to","head . "" xxunk y xxmaj sigue "" is a lovely salsa influenced track which again is so catchy will drive you nuts . xxmaj but my favourite track is the perfectly executed "" xxunk a xxmaj poco "" , xxmaj estopa 's view on smoking . xxmaj taking from the point of a smoker it describes his feelings and why he does it and what it is doing it to him"
3,"xxmaj oddly , the stations seemed to play "" dogs "" now and then , even though it was not released as a single . "" give a xxmaj hand , xxmaj take a xxmaj hand "" was an older song from 1969 that the xxmaj gibbs rerecorded after xxmaj the xxmaj staple xxmaj singers did well with it . xxmaj the song styles waver through soft , sensual ballads like ""","oddly , the stations seemed to play "" dogs "" now and then , even though it was not released as a single . "" give a xxmaj hand , xxmaj take a xxmaj hand "" was an older song from 1969 that the xxmaj gibbs rerecorded after xxmaj the xxmaj staple xxmaj singers did well with it . xxmaj the song styles waver through soft , sensual ballads like "" xxunk"
4,end 1996 xxup f250 xxbos xxmaj it works . xxbos xxmaj the only solid white leather strap i could find that did n't have any ornamentation . xxmaj it is of average quality but for the price i was n't expecting much . xxmaj it looks nice and is mostly comfortable on my black xxmaj ibanez xxup xxunk . xxbos very good xxbos i went to the local premiere of xxmaj mary,1996 xxup f250 xxbos xxmaj it works . xxbos xxmaj the only solid white leather strap i could find that did n't have any ornamentation . xxmaj it is of average quality but for the price i was n't expecting much . xxmaj it looks nice and is mostly comfortable on my black xxmaj ibanez xxup xxunk . xxbos very good xxbos i went to the local premiere of xxmaj mary xxmaj


In [9]:
dls_lm.bs

16

In [10]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [11]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(43600, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(43600, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=43600, bias=True)
    (output_dp): RNNDropout()
  )
)

In [12]:
TEXT = "Amazing product"
N_WORDS = 30
N_SENTENCES = 2

In [13]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

Amazing product and performance = World of Tomorrow , the United States and Canada , is the fifth most expensive and most expensive electronic product of all
Amazing product by oxidized produce produced in the video product . This is the product of a class of products produced in the production of 2D products . This


#### Train only the head of the model while the body stays frozen.

In [14]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.463609,4.205848,0.272985,67.077438,34:35


In [15]:
learn.save('1epoch-amazon-reviews-sample')

Path('../data/sample/models/1epoch-amazon-reviews-sample.pth')

In [16]:
# learn = learn.load('1epoch')

#### Fine-tune the model after unfreezing

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time


In [18]:
learn.save_encoder('finetuned-rt')

In [15]:
TEXT = "Good product"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [16]:
print("\n".join(preds))

Good product and high quality . A great tool for watering trees and other areas . The line works great for me . We do n't want to get a heavy duty hose to get the job done
Good product , works as advertised . Cuts like a charm . This is a great value . i have a very good yard , and i really like this stuff . It 's pretty sturdy .


In [17]:
TEXT = "Great product"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [18]:
print("\n".join(preds))

Great product . This is a very nice hose and i will be purchasing another . i have had one of these for years . i attached it to my Craftsman Home Depot hose . i have
Great product . It 's been around 3 months and i have not had a problem with bugs getting zapped . The dog has been trapped in it and maybe is not doing much of an job .


In [21]:
TEXT = "Good product"
N_WORDS = 5
N_SENTENCES = 1
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [22]:
print("\n".join(preds))

Good product . Easy to clean


In [25]:
TEXT = "Great product"
N_WORDS = 30
N_SENTENCES = 5
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [26]:
print("\n".join(preds))

Great product and price . i love this little grill . i will get another one this now and the second one is still quite heavy . The handle is
Great product . Good price . You get what you pay for . It 's exactly what it 's supposed to do . It 's well built
Great product and great price . Thanks i got this for my Weber Genesis 1 000 HP cooking grill . The grill was well
Great product Great Product . Arrived in one day . Thanks . We have two cats , Cats , moths and other mosquitoes .
Great product and price Glad to find these for a Weber . They work well , but have been too thin to hold them in place .


In [27]:
TEXT = "Poor service"
N_WORDS = 30
N_SENTENCES = 5
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [28]:
print("\n".join(preds))

Poor service . Holds up well . No problems . No leaks . This tool really works . It makes edging easier and easier .
Poor service product Beware . Bought a third house because the first one was n't last . It 's been working fine for years , but i guess
Poor service in a garden garden . No problem one with all these brass nozzles . The cheap plastic ones at the Gilmour store are a pain to use
Poor service for the money . But this is my 3rd season of using this . It is very well built and lightweight . The mounting is very sturdy
Poor service in my neighborhood . This thing just sprays water . You need to pump the water every few feet and you do n't have to pound it out


In [31]:
TEXT = "Does n't work"
N_WORDS = 30
N_SENTENCES = 5
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [32]:
print("\n".join(preds))

Does n't work as well as i hoped . It is very hard to set up . Use gloves for gloves when handling it . Great product .
Does n't work . 

 Oh , i do n't know what to say about a hose i bought as a hose . This is the only hose . i can
Does n't work for me . Very happy with the product . This is the best hummingbird feeder i have tried . Mine is made of glass , and
Does n't work very well . This unit cuts quite a bit of dust . But it does n't attract much more bugs than other ones . i would recommend
Does n't work well . Also , this is the second one I 've purchased . The first one lasted 2 years and the last one lasted 9 years


In [33]:
TEXT = "I am not satisfied"
N_WORDS = 30
N_SENTENCES = 5
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [34]:
print("\n".join(preds))

i am not satisfied with this product . It works , as well as most other reviewers have said , the handle high and the large wire just tighten . It never
i am not satisfied with this product , and it 's very cheaply made and does nt leak . It is VERY flexible and the unit is heavy and the hose feels
i am not satisfied with this feature . It is a little hard to push , but it is light and compact . It is a little bigger than i expected ,
i am not satisfied with this product . It is very well made and requires the ability to cut the grass without the noise and noise . i would recommend this product to
i am not satisfied with the product , purchased this item because i was not looking at it . It was hard to read the directions and directions which are important to cover


In [41]:
TEXT = "This mobile cover"
N_WORDS = 30
N_SENTENCES = 5
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [42]:
print("\n".join(preds))

This mobile cover is either a little flimsy and a little pricey . It is much lighter than a standard sprayer . It feels durable and is easy to use .
This mobile cover is very easy to assemble . It is simple , well made and requires a little effort to assemble . It works well , with no effort at
This mobile cover works great for my small backyard of home . i have several large suburban - room areas that stick into my house and i have a lot of clippings and
This mobile cover is perfect for our Home Depot store . It is manufactured for Christmas and has held up well . i expect it to last a long
This mobile cover for the tailgating , camping , camping trips and camping trips , is easy to manage , and it is easy to travel with a steak or grill .


In [47]:
TEXT = "Different product received"
N_WORDS = 30
N_SENTENCES = 5
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [48]:
print("\n".join(preds))

Different product received in my opinion . Fits very well . Really like it so far . Works great , needs to be cleaned , and does n't leak
Different product received . Not the one i was expecting . The second one was shipped immediately . The price was the worst price on the lower priced spools at
Different product received my Black & Decker Edge Hog . It worked well and was easy to use . i use it to trim my shrubs and trees
Different product received . This is my first bug zapper and I 've installed it on the Flowtron brand unit . It kills bugs , even with the
Different product received by Amazon for a great product . These are pretty good . i really like them , just how they last . We got this


In [51]:
TEXT = "There is little bit less space in this table"
N_WORDS = 50
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.3) 
         for _ in range(N_SENTENCES)]

In [52]:
print("\n".join(preds))

There is little bit less space in this table than the larger Weber Kettle Grill . The cooking grate is very close to the vents on the bottom of the kettle . The vents on the bottom allow you to close the vents and the vents on the bottom vents allow you to control
There is little bit less space in this table than the other one i have . It 's a bit smaller than the larger size , but it 's just a little larger than the larger size . It 's just a little larger than the larger size , but it 's so much easier to clean
